In [34]:
from bs4 import BeautifulSoup
import pandas as pd
import concurrent
import requests
import os
import os.path
import glob

In [35]:
def do_scrap(link, category):
    key = '1783319260d70f84da21868ce0fd6207'

    payload = {'api_key': key, 'url': link, 'keep_headers': 'true'}
    head = {
        'authority': 'api.tracker.gg',
        'method': 'GET',
        'path': '/api/v1/valorant/lfg/search?region=any&playlist=competitive&skill=any',
        'scheme': 'https',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'accept-encoding': 'gzip, deflate, br',
        'accept-language': 'nb-NO,nb;q=0.9,no;q=0.8,en-US;q=0.7,en;q=0.6,ru;q=0.5',
        'cache-control': 'max-age=0',
        'cookie': '__cflb=02DiuFQAkRrzD1P1mdkJhfdTc9AmTWwYkL5dhc6BaWz7E; X-Mapping-Server=s14; __cf_bm=eCW52EohIqvzfugOyKvNvo4spvffbOwnBy5YZBvh4gQ-1653777883-0-AYz5PvgOpqpL7/UCjWN7/dokINYUw/EV1D4T6eY5n9YecMF2lY0M4TWWdDyySEalQ3lzxua3IYwD9KUjzZtmFdM=',
        'sec-ch-ua': 'Not A;Brand";v="99", "Chromium";v="101", "Google Chrome";v="101',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': 'macOS',
        'sec-fetch-dest': 'document',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-site': 'none',
        'sec-fetch-user': '?1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.0.0 Safari/537.36'
        }
    count = 0
    path = f'excel_dir/{category}.csv'
    if not os.path.exists(path):
        while True:
            response = requests.get('http://api.scraperapi.com', params=payload, headers=head)
            if response.status_code == 200:
                print(f'Success:',response.status_code)
                return response
            else:
                print(f'Failure:',response.status_code)
                count += 1
                if count >= 2:
                    return None
                continue
    else:
        print(f'\t{category} has already been scrapped')
        return None
#200	Successful response. - ScraperAPI
#400    Bad request is sent when the server cannot understand the request sent by the client. - Requests
#401    Unauthorized is sent whenever fulfilling the requests requires supplying valid credentials. - Requests
#403    Forbidden means that the server understood the request but will not fulfill it. - Requests
#404	Page requested does not exist. - ScraperAPI
#410	Page requested is no longer available. - ScraperAPI
#500	After retrying for 60 seconds, the API was unable to receive a successful response. - ScraperAPI
#429	You are sending requests too fast, and exceeding your concurrency limit. - ScraperAPI
#403	You have used up all your API credits. - ScraperAPI

In [36]:
def write(response, category):
# Write html file from data scrapped
    with open(f"html_dir/{category}.html","w",encoding="utf-8")as html_file:
        html_file.write(response.text)

In [37]:
def read(category):    
    # Read html file from scrapped data
    with open(f"html_dir/{category}.html","r",encoding='utf-8')as html_file:
        output=html_file.read()
        
        soup = BeautifulSoup(output, 'lxml')
        
        products = soup.find_all('article', class_ = 'ads__unit')
            
        product_dict = {}
        for product in products:
            pass
            image = product.find('img', class_= 'img-format__img')['src']
            link = product.find('a', class_ = 'ads__unit__link')['href']
            
            price = product.find('div', class_= 'ads__unit__img__ratio__price').text
            if price == 'Til salgs' or price == 'Gis bort' or price == 'Ønskes kjøpt':
                price = None
            else:
                price = price.replace('kr', '')
                price = int(''.join(char for char in price if char.isalnum()))

            payed = product.find('span', class_ = 'status status--sponsored u-mb8')
            name = product.find('a', class_ = 'ads__unit__link').text
            location = None
            age = None
            if payed == None:
                age = product.find('div', class_ = 'ads__unit__content__details').div.text
                location = product.find('div', class_ = 'ads__unit__content__details')
                location = location.find_next()
                location = location.find_next().text

            itemnumber = product.find('a', class_ = 'ads__unit__link')['id']
            product_dict[itemnumber] = [name, price, age, location, image, link, 'finn', category]       
      
        
        return product_dict

In [45]:
def finn_scrap(links):

    # Check if we have the correct directories
    if not os.path.exists('html_dir'):
        os.mkdir('html_dir')
    if not os.path.exists('excel_dir'):
        os.mkdir('excel_dir')

    for link in links:
        # Category on the item in our current link
        part_1 = link.split('=')[2].split('&')[0].replace('+', '_')
        #part_2 = link.split('=')[1].split('&')[0]
        part_2 = link.split('page=')[1].split('&')[0]
        category = part_1 + '_' + part_2

        print(category)
        # Write down the data we get from scrapping
        scrapped = do_scrap(link, category)
        if scrapped != None:
            print('\tScrap: Success!')
            print('\tWriting...')
            write(scrapped, category)
            print('\tReading...')
            current = read(category)
            # Used for columns in excel
            col = ['name', 'price', 'age', 'location', 'image', 'link', 'store', 'category']
            # Create dataframe
            df = pd.DataFrame(current.values(), columns = col)
            #df = df[df['name'].apply(lambda x: '1070' in x)]
            #df = df.loc[df['price']<4000]
            
            df.to_csv(f'excel_dir/{category}.csv')
            print(f'\tScrapped: {category}')

In [47]:
links = []
pages = 9
for x in range(1,pages+1):
        links.append(f'https://www.finn.no/bap/forsale/search.html?location=0.22030&location=0.22054&page={x}&product_category=2.93.3907.79&sort=RELEVANCE')

In [48]:
finn_scrap(links)

1
0.22054_1
	0.22054_1 has already been scrapped
2
0.22054_2
Success: 200
	Scrap: Success!
	Writing...
	Reading...
	Scrapped: 0.22054_2
3
0.22054_3
Success: 200
	Scrap: Success!
	Writing...
	Reading...
	Scrapped: 0.22054_3
4
0.22054_4
Success: 200
	Scrap: Success!
	Writing...
	Reading...
	Scrapped: 0.22054_4
5
0.22054_5
Success: 200
	Scrap: Success!
	Writing...
	Reading...
	Scrapped: 0.22054_5
6
0.22054_6
Success: 200
	Scrap: Success!
	Writing...
	Reading...
	Scrapped: 0.22054_6
7
0.22054_7
Success: 200
	Scrap: Success!
	Writing...
	Reading...
	Scrapped: 0.22054_7
8
0.22054_8
Success: 200
	Scrap: Success!
	Writing...
	Reading...
	Scrapped: 0.22054_8
9
0.22054_9
Success: 200
	Scrap: Success!
	Writing...
	Reading...
	Scrapped: 0.22054_9


In [49]:
# Combines all pages into one file

#os.chdir("excel_dir")

extension = 'csv'
all_filenames = [i for i in glob.glob(f'excel_dir/*.{extension}')]

#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
#export to csv
combined_csv.to_csv( "combined_csv.csv", index=False, encoding='utf-8-sig')